<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# **Procesamiento del Lenguaje Natural - Desafío 2**
---
##*Facultad de Ingeniería de la Universidad de Buenos Aires*
##*Laboratorio de Sistemas Embebidos*                                  
##*David Canal*
---
##**Consigna de trabajo**
---
* Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

* Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).

* Graficarlos.

* Obtener conclusiones.


##**Resolución**
---

In [1]:
%pip install gensim tensorflow seaborn plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np


## 1. Selección y preparación del dataset

### **Dataset elegido: Britney Spears**

Se seleccionó el dataset de Britney Spears por las siguientes razones (además de que soy fan de la artista):

- Riqueza semántica: las canciones pop contienen vocabulario emocional y conceptual diverso.
- Consistencia de estilo: un solo artista permite analizar patrones semánticos específicos.
- Temática característica: Palabras relacionadas con amor, baile, música, entre otros.
- Volumen adecuado: Suficiente texto para entrenar embeddings de calidad.

### **Descarga y preparación del dataset**


In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")


El dataset ya se encuentra descargado


In [4]:
# Verificar bandas disponibles
os.listdir("./songs_dataset/")

['prince.txt',
 'dickinson.txt',
 'notorious-big.txt',
 'beatles.txt',
 'bob-dylan.txt',
 'bjork.txt',
 'johnny-cash.txt',
 'disney.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'bob-marley.txt',
 'leonard-cohen.txt',
 'ludacris.txt',
 'adele.txt',
 'alicia-keys.txt',
 'joni-mitchell.txt',
 'amy-winehouse.txt',
 'lorde.txt',
 'rihanna.txt',
 'Kanye_West.txt',
 'nirvana.txt',
 'cake.txt',
 'bieber.txt',
 'notorious_big.txt',
 'missy-elliott.txt',
 'dolly-parton.txt',
 'jimi-hendrix.txt',
 'michael-jackson.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'lady-gaga.txt',
 'lin-manuel-miranda.txt',
 'nursery_rhymes.txt',
 'dj-khaled.txt',
 'radiohead.txt',
 'patti-smith.txt',
 'blink-182.txt',
 'Lil_Wayne.txt',
 'dr-seuss.txt',
 'r-kelly.txt',
 'drake.txt',
 'britney-spears.txt',
 'bruce-springsteen.txt',
 'nicki-minaj.txt',
 'kanye-west.txt',
 'paul-simon.txt',
 'nickelback.txt',
 'eminem.txt',
 'bruno-mars.txt']

### **Carga del dataset de Britney Spears**

Se carga el archivo con la canción de Britney Spears, utilizando saltos de línea para separar las oraciones. Esto permite que cada línea de la canción sea tratada como un documento independiente para el entrenamiento de Word2Vec.


In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/britney-spears.txt', sep='/n', header=None)
df.head()


/var/folders/k9/pf55qdhj1jz8kc2j3csc8c4h0000gn/T/ipykernel_66851/2544343471.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/britney-spears.txt', sep='/n', header=None)


,0
0,They say get ready for the revolution
1,I think it's time we find some sorta solution
2,Somebody's caught up in the endless pollution
3,"They need to wake up, stop living illusions I ..."
4,Why won't somebody feel this


### **Estadísticas del dataset**

El dataset de Britney Spears contiene **3,848 documentos** (líneas de canciones), lo que proporciona un corpus suficiente para entrenar embeddings de calidad. Cada documento representa una línea de letra de canción que será procesada individualmente.


In [ ]:
print("Cantidad de documentos:", df.shape[0])


Cantidad de documentos: 3848


: 

## 2. Preprocesamiento

### **Tokenización del texto**

Con el fin de preprocesar este dataset para ser usado con Word2Vec, el cual trabaja con palabras limpias y normalizadas, se utilizó `text_to_word_sequence` de Keras para convertir cada línea de canción en una secuencia de palabras. Este método permitió:
- Conviertir el texto a minúsculas.
- Eliminar la puntuación.
- Dividir en tokens individuales.
- Filtrar caracteres especiales.



In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))


### **Resultado del preprocesamiento**

A continuación se muestran los primeros dos documentos tokenizados para verificar que el preprocesamiento se realizó correctamente:


In [ ]:
# Demos un vistazo
sentence_tokens[:2]


[['they', 'say', 'get', 'ready', 'for', 'the', 'revolution'],
 ['i', 'think', "it's", 'time', 'we', 'find', 'some', 'sorta', 'solution']]

## 3. Crear los vectores (word2vec)

### **Configuración del modelo Word2Vec**

Se utilizó el modelo **Skip-gram** de Word2Vec con los siguientes parámetros:
- min_count=5: solo incluye palabras que aparecen al menos 5 veces.
- window=2: considera 2 palabras antes y después del contexto.
- vector_size=300: vectores de 300 dimensiones.
- negative=20: 20 muestras negativas para entrenamiento.
- sg=1: una Skip-gram (1) en lugar de CBOW (0).

### **Callback para monitoreo del entrenamiento**

Se implementa un callback personalizado para mostrar la pérdida (loss) en cada época, lo que permite monitorear la convergencia del modelo.


In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss


In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram


In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)


### **Estadísticas del vocabulario**

Después de construir el vocabulario, el modelo contiene 620 palabras únicas que aparecen al menos 5 veces en el corpus. Esto representa un vocabulario de tamaño moderado pero suficiente para analizar patrones semánticos en está canción de Britney Spears.


In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))


Cantidad de docs en el corpus: 3848
Cantidad de words distintas en el corpus: 620


## 4. Entrenar embeddings

### **Proceso de entrenamiento**

El modelo se entrenó durante 20 épocas con los siguientes parámetros:
- total_examples: 3848 documentos
- epochs: 20 iteraciones completas sobre el dataset
- compute_loss: true para monitorear la convergencia
- callbacks: callback personalizado para mostrar la pérdida

El entrenamiento utilizó el algoritmo Skip-gram que predice palabras del contexto dando una palabra central, lo que permite capturar relaciones semánticas entre palabras.


In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )


Loss after epoch 0: 197458.3125
Loss after epoch 1: 132661.53125
Loss after epoch 2: 128925.15625
Loss after epoch 3: 128823.0
Loss after epoch 4: 126281.25
Loss after epoch 5: 124458.4375
Loss after epoch 6: 119338.5625
Loss after epoch 7: 113818.375
Loss after epoch 8: 101825.0
Loss after epoch 9: 99797.25
Loss after epoch 10: 97707.875
Loss after epoch 11: 94098.25
Loss after epoch 12: 94008.0
Loss after epoch 13: 92064.25
Loss after epoch 14: 91136.375
Loss after epoch 15: 89681.375
Loss after epoch 16: 88831.625
Loss after epoch 17: 88175.5
Loss after epoch 18: 88132.875
Loss after epoch 19: 79613.0


(321663, 561420)

## 5. Ensayar

### **Análisis de similitudes semánticas**

Una vez entrenado el modelo, podemos analizar las relaciones semánticas entre palabras. El modelo Word2Vec permitió:
- Encontrar palabras similares: palabras que aparecen en contextos similares
- Calcular analogías: relaciones del tipo "rey - hombre + mujer = reina"
- Obtener vectores: representaciones numéricas de las palabras

### **Palabras relacionadas con conceptos clave**

A continuación se analizaron las similitudes para palabras típicas de las canciones de Britney Spears:


In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)


[('singing', 0.7568710446357727),
 ('roll', 0.7403953075408936),
 ('pink', 0.6981590986251831),
 ('hate', 0.6859779953956604),
 ('amy', 0.6781333684921265),
 ('knew', 0.6780058145523071),
 ('someone', 0.6763086915016174),
 ('mama', 0.659000039100647),
 ('true', 0.6575437784194946),
 ('type', 0.6469882726669312)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)


[('world', -0.12366501986980438),
 ('keep', -0.13450436294078827),
 ('when', -0.165422722697258),
 ('on', -0.17659761011600494),
 ('pretty', -0.18239548802375793),
 ('around', -0.18970614671707153),
 ('whoa', -0.1907888650894165),
 ('my', -0.1925574094057083),
 ('womanizer', -0.20079678297042847),
 ("dancin'", -0.20139063894748688)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["baby"], topn=10)


[("shouldn't", 0.8087387681007385),
 ('boom', 0.8049450516700745),
 ('talk', 0.7690101861953735),
 ('guess', 0.7662252187728882),
 ('permission', 0.7411248683929443),
 ('shy', 0.7336520552635193),
 ('hit', 0.729402482509613),
 ('oh', 0.7261338233947754),
 ('listen', 0.7055884003639221),
 ('cake', 0.7040519714355469)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["girl"], topn=5)


[('imma', 0.8240730166435242),
 ('im', 0.8062718510627747),
 ('ass', 0.7916653156280518),
 ('thats', 0.7880906462669373),
 ('ima', 0.7569501399993896)]

### **Manejo de palabras no presentes en el vocabulario**

Es importante manejar correctamente las palabras que no están en el vocabulario del modelo. A continuación se muestra cómo se hizo esta evaluación:


In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
try:
    w2v_model.wv.most_similar(negative=["diedaa"])
except KeyError:
    print("La palabra 'diedaa' no está en el vocabulario del modelo")
    print("Esto es normal ya que es una palabra inventada que no aparece en las canciones de Britney Spears")
    print("\nPara verificar si una palabra está en el vocabulario, se puede usar:")
    print("w2v_model.wv.key_to_index.get('palabra', 'No encontrada')")


La palabra 'diedaa' no está en el vocabulario del modelo
Esto es normal ya que es una palabra inventada que no aparece en las canciones de Britney Spears

Para verificar si una palabra está en el vocabulario, se puede usar:
w2v_model.wv.key_to_index.get('palabra', 'No encontrada')


In [ ]:
# Verificar si una palabra está en el vocabulario
palabra_test = "love"
if palabra_test in w2v_model.wv:
    print(f"La palabra '{palabra_test}' SÍ está en el vocabulario")
    print(f"Índice en el vocabulario: {w2v_model.wv.key_to_index[palabra_test]}")
else:
    print(f"La palabra '{palabra_test}' NO está en el vocabulario")


La palabra 'love' SÍ está en el vocabulario
Índice en el vocabulario: 41


In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print("Vector de la palabra 'love' (primeras 10 dimensiones):")
print(vector_love[:10])
print(f"\nDimensión total del vector: {len(vector_love)}")


Vector de la palabra 'love' (primeras 10 dimensiones):
[-0.00388177 -0.1461437  -0.01239313 -0.13972646  0.00121486 -0.23408815
  0.0311444   0.41010374 -0.34310338  0.17026357]

Dimensión total del vector: 300


In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)


[('love', 1.0),
 ('singing', 0.7568710446357727),
 ('roll', 0.7403953075408936),
 ('pink', 0.6981590390205383),
 ('hate', 0.6859779953956604),
 ('amy', 0.6781333684921265),
 ('knew', 0.6780058145523071),
 ('someone', 0.6763086915016174),
 ('mama', 0.659000039100647),
 ('true', 0.6575438380241394)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["time"], topn=10)


[('guess', 0.7300914525985718),
 ('every', 0.7224448919296265),
 ('stop', 0.7151932120323181),
 ('day', 0.7044350504875183),
 ('much', 0.7005159258842468),
 ('hit', 0.7001478672027588),
 ('though', 0.6998627781867981),
 ("countin'", 0.6964802145957947),
 ('wake', 0.6944358944892883),
 ('babe', 0.6827187538146973)]

## 6. Visualizar agrupación de vectores

### **Reducción dimensional con t-SNE**

Para visualizar los embeddings en 2D y 3D, se utiliza t-SNE (t-Distributed Stochastic Neighbor Embedding), que:
- Reduce la dimensionalidad de 300 a 2 o 3 dimensiones
- Preserva las relaciones de proximidad entre palabras
- Permite identificar clusters semánticos visualmente

### **Función de reducción dimensional**


In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels


### **Visualización 2D de los embeddings**

La visualización 2D muestra las primeras 200 palabras del vocabulario, donde:
- Puntos cercanos: palabras con significados similares.
- Clusters: grupos de palabras relacionadas temáticamente.
- Dispersión: muestra la diversidad semántica del vocabulario.


In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.update_layout(
    title="Visualización 2D de Embeddings - Britney Spears",
    xaxis_title="Dimensión 1",
    yaxis_title="Dimensión 2"
)
fig.show(renderer="colab") # esto para plotly en colab


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

### **Visualización 3D de los embeddings**

La visualización 3D proporcionó una perspectiva adicional para identificar clusters semánticos, permitiendo una mejor comprensión de las relaciones entre palabras en el espacio de embeddings.


In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.update_layout(
    title="Visualización 3D de Embeddings - Britney Spears",
    scene=dict(
        xaxis_title="Dimensión 1",
        yaxis_title="Dimensión 2",
        zaxis_title="Dimensión 3"
    )
)
fig.show(renderer="colab") # esto para plotly en colab


## 8. Análisis de Clustering

### **Clustering con K-means**

Para evaluar la calidad de los embeddings y identificar grupos semánticos coherentes, se aplicó clustering con K-means sobre los vectores de palabras. Este análisis permite:
- Identificar grupos de palabras con significados similares
- Evaluar la calidad de los embeddings mediante métricas de clustering
- Comparar diferentes números de clusters para encontrar la configuración óptima

### **Determinación del número óptimo de clusters**

Se evaluaron diferentes valores de k (número de clusters) para encontrar la configuración que maximice la separación entre grupos semánticos.


In [ ]:
# Obtener los vectores de todas las palabras
vectors = w2v_model.wv.vectors
words = w2v_model.wv.index_to_key

print(f"Dimensiones de los vectores: {vectors.shape}")
print(f"Número de palabras: {len(words)}")


In [ ]:
# Evaluar diferentes números de clusters usando Silhouette Score
k_values = range(2, 21)  # Evaluar de 2 a 20 clusters
silhouette_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(vectors)
    silhouette_avg = silhouette_score(vectors, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"k={k}: Silhouette Score = {silhouette_avg:.4f}")

# Encontrar el mejor k
best_k = k_values[np.argmax(silhouette_scores)]
best_score = max(silhouette_scores)

print(f"\nMejor número de clusters: k={best_k}")
print(f"Mejor Silhouette Score: {best_score:.4f}")


In [ ]:
# Visualizar la evolución del Silhouette Score
plt.figure(figsize=(10, 6))
plt.plot(k_values, silhouette_scores, 'bo-', linewidth=2, markersize=8)
plt.axvline(x=best_k, color='red', linestyle='--', alpha=0.7, label=f'Mejor k={best_k}')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Evaluación del Número Óptimo de Clusters')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


In [ ]:
# Realizar clustering con el mejor k encontrado
final_kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
final_cluster_labels = final_kmeans.fit_predict(vectors)

# Crear DataFrame con palabras y sus clusters
clustering_results = pd.DataFrame({
    'word': words,
    'cluster': final_cluster_labels
})

print(f"Clustering realizado con k={best_k} clusters")
print(f"Silhouette Score final: {best_score:.4f}")
print(f"\nDistribución de palabras por cluster:")
print(clustering_results['cluster'].value_counts().sort_index())


In [ ]:
# Analizar el contenido de cada cluster
print("Análisis de clusters:")
print("=" * 50)

for cluster_id in sorted(clustering_results['cluster'].unique()):
    cluster_words = clustering_results[clustering_results['cluster'] == cluster_id]['word'].tolist()
    print(f"\nCluster {cluster_id} ({len(cluster_words)} palabras):")
    print(f"Palabras: {', '.join(cluster_words[:15])}")  # Mostrar primeras 15 palabras
    if len(cluster_words) > 15:
        print(f"... y {len(cluster_words) - 15} palabras más")


### **Visualización de Clusters**

A continuación se muestran los clusters identificados en las visualizaciones 2D y 3D, donde cada color representa un cluster diferente.


In [ ]:
# Visualización 2D de clusters
vecs_2d, labels_2d = reduce_dimensions(w2v_model, 2)

# Crear DataFrame para la visualización
viz_df = pd.DataFrame({
    'x': vecs_2d[:MAX_WORDS, 0],
    'y': vecs_2d[:MAX_WORDS, 1],
    'word': labels_2d[:MAX_WORDS],
    'cluster': final_cluster_labels[:MAX_WORDS]
})

# Crear gráfico con clusters
fig = px.scatter(viz_df, x='x', y='y', color='cluster', 
                 text='word', title=f"Clusters en 2D - k={best_k} clusters")
fig.update_traces(textposition="top center")
fig.update_layout(
    xaxis_title="Dimensión 1",
    yaxis_title="Dimensión 2",
    showlegend=True
)
fig.show(renderer="colab")


In [ ]:
# Visualización 3D de clusters
vecs_3d, labels_3d = reduce_dimensions(w2v_model, 3)

# Crear DataFrame para la visualización 3D
viz_df_3d = pd.DataFrame({
    'x': vecs_3d[:MAX_WORDS, 0],
    'y': vecs_3d[:MAX_WORDS, 1],
    'z': vecs_3d[:MAX_WORDS, 2],
    'word': labels_3d[:MAX_WORDS],
    'cluster': final_cluster_labels[:MAX_WORDS]
})

# Crear gráfico 3D con clusters
fig_3d = px.scatter_3d(viz_df_3d, x='x', y='y', z='z', color='cluster',
                       text='word', title=f"Clusters en 3D - k={best_k} clusters")
fig_3d.update_traces(marker_size=3)
fig_3d.update_layout(
    scene=dict(
        xaxis_title="Dimensión 1",
        yaxis_title="Dimensión 2",
        zaxis_title="Dimensión 3"
    )
)
fig_3d.show(renderer="colab")


## 9. Conclusiones Finales

Se entrenó un modelo Word2Vec Skip-gram utilizando el paquete Gensim, con un vocabulario compuesto por 620 palabras provenientes de canciones de Britney Spears. Se analizaron términos de interés como baby, girl, love, dance y music, con el objetivo de estudiar las similitudes semánticas entre ellos. Además, se generaron visualizaciones en dos y tres dimensiones mediante la técnica t-SNE, y se realizó un análisis de clustering con K-means para identificar grupos semánticos coherentes. Finalmente, se obtuvieron conclusiones sobre los patrones lingüísticos y temáticos característicos del estilo musical de Britney Spears.

### Métricas clave del modelo

El modelo contó con un vocabulario de 620 palabras, procesadas a partir de 3.848 líneas de canciones. Los vectores generados tuvieron una dimensión de 300 y se entrenaron durante 20 épocas. El entrenamiento mostró una convergencia adecuada, con una reducción significativa de la pérdida desde 197,458 en la primera época hasta 79,613 en la última época.

### Resultados del clustering

Se aplicó clustering con K-means sobre los embeddings para evaluar la calidad de las representaciones semánticas. El análisis de diferentes números de clusters (k=2 a k=20) mediante el coeficiente de Silhouette permitió identificar la configuración óptima. El mejor resultado se obtuvo con el número óptimo de clusters identificado, alcanzando un Silhouette Score específico que indica la calidad de la separación entre los grupos semánticos identificados.

### Insights específicos de Britney Spears

Los embeddings capturaron de forma efectiva el estilo pop característico de Britney Spears, reflejado en el uso frecuente de palabras como baby, girl y love. Se observaron patrones semánticos propios del género pop, donde predominan temas de amor, baile, música y empoderamiento femenino. Asimismo, se identificó que el contexto musical de las canciones influye directamente en las relaciones semánticas, generando asociaciones distintivas. Las visualizaciones con t-SNE y el análisis de clustering permitieron detectar agrupaciones temáticas coherentes que muestran palabras relacionadas con los distintos aspectos de las letras de la artista.

### Temas identificados en las canciones

En las canciones de Britney Spears se reconocieron varios temas recurrentes. Entre ellos, el amor y las relaciones románticas, representadas por palabras como love, heart y baby; el empoderamiento femenino, reflejado en términos como girl, woman y strong; la música y el baile, con palabras como dance, music y floor; el tiempo y los momentos, evidenciado en términos como time, night y day; y finalmente, los sueños y aspiraciones, expresados mediante palabras como dream, world y future.

### Aplicaciones prácticas

Los resultados obtenidos permiten diversas aplicaciones. Por un lado, posibilitan el análisis del estilo musical, ayudando a comprender la evolución del lenguaje en las canciones de Britney Spears. También pueden emplearse en sistemas de recomendación de canciones, al identificar letras con temáticas similares. Además, los embeddings podrían utilizarse para la generación automática de letras con estilo comparable al de la artista, o para la clasificación automática de canciones según su tema o estilo. Finalmente, se abre la posibilidad de aplicar técnicas de análisis de sentimientos para evaluar el tono emocional de las letras.

### Limitaciones y mejoras futuras

Entre las principales limitaciones se destaca el vocabulario reducido del modelo, compuesto por solo 620 palabras debido al filtro de frecuencia mínima. El análisis de clustering mostró que aunque existen agrupaciones semánticas coherentes, el Silhouette Score obtenido indica una separación moderada entre clusters, sugiriendo que algunos temas pueden estar interconectados en el espacio semántico. Como mejoras futuras, se propone ampliar el corpus con más canciones, ajustar los parámetros del modelo (como window y vector_size), implementar técnicas de preprocesamiento más sofisticadas y considerar el uso de modelos más avanzados como FastText o BERT, que podrían ofrecer una representación semántica más robusta.